In [ ]:
import cudf, cuml, cugraph, cuxfilter
import json
from pandas.io.json import json_normalize 
import pandas as pd 
import time

In [ ]:
#Fase de extracción y lectura con Rapids

#extracción y lectura con Pandas
pd_Pandas = pd.read_csv('df_data.csv',
                        names='uno')

In [ ]:
cudf_Pandas = pd_Pandas
cudf_Pandas

In [ ]:
#Fase Transformation
#Dataset more understandable

import json
#create format json for later create dataframe
lista = list(cudf_Pandas['u'])
a = []

for i in range(0,len(lista)):
    d = lista[i].split('.gz:')
    nuevo = d[1]
    a.append(json.loads((nuevo)))
    
#Creo DataFrame con Pandas
pdPandas = pd.DataFrame(a)

# Convierto todo el dataframe a str
nuevo_pdPandas = pdPandas.applymap(str)

#Se crear un dataframe con cudf(propia de Rapids)
gdf = cudf.DataFrame.from_pandas(nuevo_pdPandas)

#conviero para pandas dataframe(propia de Rapids)
gdfPD = gdf.to_pandas()

#solo conservo los datos de la columna username
#mayor a 0
snNull = gdfPD[gdfPD.username.map(len) > 0]

#elimino username dicc y opencampus
snDicc = snNull[snNull.username != 'dicc']
snCampus = snDicc[snDicc.username != 'opencampus']

#convieto para cudf dataframe
gdf_new = cudf.DataFrame.from_pandas(snCampus)

#eliminar filas con los datos perdidos
gdf_new.dropna(axis=0, how='any')

#Eliminar columnas que no tiene mucha relevancia
gdf_new.drop_column('accept_language') #columna si relevancia
gdf_new.drop_column('name') #muchos datos faltantes
gdf_new.drop_column('agent') #columna si relevancia
gdf_new.drop_column('page') #muchos datos faltantes
gdf_new.drop_column('session') #muchos datos faltantes
gdf_new.drop_column('event_type') #columna si relevancia


#columna ip sea formato int
gdf_new['ip'] = gdf_new['ip'].str.ip2int()

#eliminar filas con datos perdidos
gdf_new.dropna(axis=0, how='any')

#soltar duplicados
gdf_new.sort_values('username') 
gdf_new.drop_duplicates(subset='username', keep=False, inplace=False, ignore_index=False)

#resetear index
gdf_new = gdf_new.reset_index()
gdf_new.drop_column('index')

#eliminar filas con datos perdidos
gdf_new.dropna(axis=0, how='any')

#dividir columna time
time = gdf_new['time']
time.str.normalize_spaces()
newtime = time.str.split(pat = "-")

day = newtime[2]
day.str.normalize_spaces()
daytime = day.str.split(pat = 'T')
daytime.drop_column(1)
daytime.columns = ['day']

timenew = newtime.join(daytime, how='left')
timenew.drop_column(2)
timenew.columns = ['year','month','day']

#Unir todo al dataframe general
gdf_new = gdf_new.join(timenew)
gdf_new.drop_column('time')

gdf_new['year'] = gdf_new['year'].astype('int32')
gdf_new['month'] = gdf_new['month'].astype('int32')
gdf_new['day'] = gdf_new['day'].astype('int32')

In [ ]:
#information with RAPIDS of all Dataset
#print('Names columns:',newData.columns)
#print('Dataframe empty yes or no:',newData.empty)
##print('Dimension DataFrame:',newData.shape)
#print('Verificar valores nulos por cada columna:',newData.isnull().sum())
print('tipos de datos de cada columna', gdf_new.dtypes)
print('Information DATA general again with RAPIDS:')
gdf_new.head(2)

In [ ]:
newContext = gdf_new['context'].tolist()

b=[]
for i in range(0,len(newContext)):
    d = newContext[i].split(',')
    nuevo = d[1]
    b.append(nuevo)

gdfContext = cudf.DataFrame()
gdfContext['General'] = b
gdfContext = cudf.DataFrame(gdfContext['General'].str.split())
gdfContext.drop_column(0)
gdfContext.columns=['userid_orgid']

newData = gdf_new.join((gdfContext), gdf_new.drop_column('context'),how ='left')
newData = newData.drop_duplicates(subset=['username'], keep='first')
newData.head(2)

In [ ]:
#Proceso columna event transformar
newEvent = newData['event'].tolist()

valor = []
newValor = []
for i in range(0,len(newEvent)):
    if (type(newEvent[i]==str)):
        try:
            valor = json.loads(newEvent[i])
        except ValueError as es:
            #print('llego')
            newEvent[i] = valor
            newValor.append(newEvent[i])

eventPd = pd.DataFrame(newValor)
eventPd.head()

nuevovalorEventPost = pd.concat([eventPd.drop(['POST'], axis=1),
           eventPd.POST.apply(pd.Series)], axis=1)
nuevovalorEventPost